In [1]:
#%reset -f
import pandas as pd
#import numpy as np
#import matplotlib.pyplot as plt
import os
import glob
#import math
from datetime import date, timedelta
#from datetime import datetime
# import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from plotly.offline import plot

In [2]:
print (os.getcwd())
os.chdir('c:\\QUIK_for_stat\\lua')

c:\Users\Administrator\repos\jupyter\plotly


In [2]:
#Определяем дату, за которую будем строить график
"""today = date.today()
# YYmmdd
d1 = today.strftime("%Y%m%d")
print("d1 =", d1)"""

#today = date.today().strftime("%Y%m%d")
#prev_day
today= (date.today() - timedelta(days=1)).strftime("%Y%m%d") #return prev date
today='20220804'

In [3]:
# функция для поиска файлов с нужной датой в соответствующей папке
def f_txtfiles(today):
    txtfiles = []
    #for file in glob.glob(".\\lua_output_txt\\*IPADDRESS*"+today+"*.txt"):
    for file in glob.glob(".\\data\\*IPADDRESS*"+today+"*.txt"):
            txtfiles.append(file)
    return txtfiles

In [4]:
# формируем лист с тексовыми файлами и данными за нужную дату
txtfiles=f_txtfiles(today=today)

In [5]:
# функия для создания датафрейма с результатами по всем рынкам
def f_df_append(txtfiles):
    file=0
    if len(txtfiles)>1:
        while (file<len(txtfiles)):
            try:
                df
            except:
                df=pd.read_csv(txtfiles[file],sep=';',header=0)
                file=+1
                #print('except file='+str(file))
  
            #print('next')
            df_temp=pd.read_csv(txtfiles[file],sep=';',header=0)
            df=df.append(df_temp)
            file=file+1
            #print('file='+str(file))
    elif len(txtfiles)==1:
        df=pd.read_csv(txtfiles[file],sep=';',header=0)  
    return df

In [6]:
# формируем датафрейм для визуализации данных исползуя функцию 
df=f_df_append(txtfiles)

In [7]:
# функция преобразования данных для визуализации 
def f_df_transform(df):
    df=df.reset_index()
    del df['index']
    df=df[df['delta_time']<10]
    df[['date','local_time']]=df[['date','local_time']].astype(str)
    df['len_local_time']=df.local_time.map(len)
    df.loc[df['len_local_time']==5,'local_time']='0'+df['local_time']
    df.loc[df['len_local_time']==4,'local_time']='00'+df['local_time']
    df.loc[df['len_local_time']==3,'local_time']='000'+df['local_time']
    df.loc[df['len_local_time']==2,'local_time']='0000'+df['local_time']
    df.loc[df['len_local_time']==1,'local_time']='00000'+df['local_time']
    del df['len_local_time']
    df['date_time']=df['date']+' '+df['local_time']
    df['date_time']=pd.to_datetime(df['date_time'],format='%Y%m%d %H%M%S')
    df=df.set_index('date_time')
    df['date_time']=df.index
    df.index.name=None
    df=df.sort_values(by=['date_time'])
    df['seconds']=df.delta_time
    df=df.drop(['delta_time'],axis=1)
    df['Market']=df.Stock
    df=df.drop(['Stock'],axis=1)
    df=df.sort_values(['Market','type'],ascending=[True,False])
    return df

In [9]:
# преобразуем данные для визуализации
df=f_df_transform(df)

#### Data Visualisation 1

In [10]:
# визуализируем данные https://plotly.com/python/reference/index/
color_map= {'FR': 'rgb(235, 52, 52)' , 'CETS': 'rgb(235, 149, 52)', 'FORTS': 'rgb(67, 52, 235)'}

fig=px.scatter(data_frame=df,x='date_time',y='seconds',title='QUIK STATISTICS'
            #,facet_col='Market'
            ,symbol='type'
            ,color='Market'
            ,hover_name='Market'
            #,hover_data=['order_num','date_time','seconds','type']
            ,hover_data={'order_num':True,'date_time':True,'seconds':True,'type':True}
            ,color_discrete_map=color_map
            #,symbol_sequence= ['circle', 'circle-open-dot', 'square','square-open-dot','diamond','diamond-open-dot']
            ,symbol_sequence= ['circle','circle-open-dot']
            #,color_discrete_sequence=['blue', 'orange', 'green', 'brown','black','yellow']
               )
# update title
fig.update_layout({'title':{'font':{'size':20},'x':0.5,'xanchor':'center'}})
# add slider
#fig.update_layout({'xaxis':{'rangeslider':{'visible':False},'autorange':True}})
#-----------------------------------------------------------------------------------------------
# drop_down buttons
# Create the buttons
dropdown_buttons = [
    {'label': "ALL MARKETS", 'method': "update", 'args': [{"visible": [True, True,True,True,True,True]}, {"title": "QUIK STATISTIC"}]},
    {'label': "CETS send_order", 'method': "update", 'args': [{"visible": [True, False, False,False,False,False]}, {"title": "CETS send_order"}]},
    {'label': "CETS kill_order", 'method': "update", 'args': [{"visible": [False, True, False, False,False,False]}, {"title": "CETS kill_order"}]},
    {'label': "FORTS send_order", 'method': "update", 'args': [{"visible": [False, False, True,False,False,False]}, {"title": "FORTS send_order"}]},
    {'label': "FORTS kill_order", 'method': "update", 'args': [{"visible": [False, False, False, True, False, False]}, {"title": "FORTS kill_order"}]},
    {'label': "FR send_order", 'method': "update", 'args': [{"visible": [False, False, False,False,True,False]}, {"title": "FR send_order"}]},
    {'label': "FR kill_order", 'method': "update", 'args': [{"visible": [False, False, False, False,False,True]}, {"title": "FR kill_order"}]},
]
# Update the figure to add dropdown menu
fig.update_layout({
  		'updatemenus': [
        {'active': 0, 'buttons': dropdown_buttons}
        ]})
#-----------------------------------------------------------------------------------------   
# legend and axis update
# create legend dictionary
my_legend={'title':{'text':'ALL MARKETS','font':{'color':'red'}}
            ,'bgcolor':'rgb(246,228,129)'
            ,'bordercolor':'black','borderwidth':1
            ,'font':{'size':16}
            ,'groupclick':'toggleitem'
            #,'orientation':'h'
            #,'yanchor':'top'
            #,'y':0.99
            #,'xanchor':'right'
            #,'x':0.2
            }

fig.update_layout({ 
                'xaxis':{'title':{'text':'Date_time','font':{'size':20}}}
                ,'yaxis':{'title':{'text':'Response to transaction in seconds','font':{'size':20}}}
                ,'legend':my_legend
                })
#-----------------------------------------------------------------------------------------         
# hover update  
''' # first method
fig.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="Rockwell"
    )
)
'''
# second method
fig.update_layout({'hoverlabel':{'bgcolor':'white','font_size':16}})
#-----------------------------------------------------------------------------------------         
# marker update  

''' # first method
fig.update_traces(marker=dict(size=10,
                              line=dict(width=1,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))
'''
# second method
fig.update_traces({'marker': {'size':10,'line':{'width':1,'color':'DarkSlateGrey'}} } )
#fig.update_traces({'marker': {'size':20,'line':{'width':1,'color':'DarkSlateGrey'}} }, {'selector':{'mode':'markers'}} )
#-----------------------------------------------------------------------------------------  
# add annotation example
'''
annotation = { 
    # set coordinates of annotation
    'xref':'paper', 'yref':'paper', 'x':0.5,'y':1.01
    # set text and format
    ,'bgcolor':'blue'
    ,'showarrow':False
    ,'text':'Server QUIK'
    ,'font':{'color':'white','family':'Courier New','size':14}
    }
'''
#annotation1={'font':{'size':16}}
#annotation2={'font':{'size':16}}
#annotation3={'font':{'size':16}}
#fig.update_layout({'annotations':[annotation]})
#fig.update_layout({'annotations':[annotation1,annotation2,annotation3]})

#-----------------------------------------------------------------------------------------  
# plot data
plot(fig)

#path=os.getcwd()+'\QUIK_STAT_html'
#plot(figure_or_data=fig,filename=path+'\QUIK_STAT_'+today+'.html',auto_open=False)

'temp-plot.html'

In [12]:
#print(fig)
#fig.to_json('fig.txt')
fig.write_json('fig.txt')
print (os.getcwd())

c:\Users\chupa\git\plotly_examples\plotly


#### Data Visualisation 2

In [13]:
'''# визуализируем данные https://plotly.com/python/reference/index/
fig = make_subplots(rows=2,cols=1, subplot_titles=['First','Second'])
fig.add_trace(go.Scatter(x=df.date_time,y=df.seconds,name='',showlegend=False),row=1,col=1)
#fig.add_trace(go.Box(x=df.Market,y=df.seconds,name='',orientation='v',showlegend=False),row=2,col=1)
fig.add_trace(go.Box(x=df.seconds,y=df.Market,name='',orientation='h',showlegend=False),row=2,col=1)
fig.update_layout({'title':{'text':'QUIK STATISTIC','x':0.5,'y':1}})
plot(fig)
'''
# визуализируем данные https://plotly.com/python/reference/index/
fig = make_subplots(rows=1,cols=2, subplot_titles=['First','Second'],shared_yaxes=True)
fig.add_trace(go.Scatter(x=df.date_time,y=df.seconds,name='',showlegend=False,mode='markers'),row=1,col=1)
#fig.add_trace(go.Violin(x=df.Market,y=df.seconds,name='',orientation='v',showlegend=False),row=1,col=2)
fig.add_trace(go.Violin(
            x=df.Market,y=df.seconds,name=''
            ,orientation='v'
            ,showlegend=False
            ,box_visible=True
            ,meanline_visible=True
            ,line_color='black'
            )
    ,row=1,col=2)
fig.update_layout({'title':{'text':'QUIK STATISTIC','x':0.5,'y':1}})
plot(fig)

'temp-plot.html'

#### Data Visualisation 3

In [15]:
# визуализируем данные https://plotly.com/python/reference/index/
'''
fig = make_subplots(rows=1,cols=3, subplot_titles=['FR','CETS','FORTS'],shared_yaxes=True)
col_num=1
for market in ['FR','FORTS','CETS']:
    df_=df[df.Market==market]
    fig.add_trace(go.Scatter(x=df_.date_time,y=df_.seconds,name=market,mode='markers'),row=1,col=col_num)
    col_num+=1
fig.update_layout({'title':{'text':'QUIK STATISTIC','x':0.5,'y':1}})
#fig.update_layout({'marker':''})
plot(fig)
'''
# визуализируем данные https://plotly.com/python/reference/index/
fig = make_subplots(rows=1,cols=3, subplot_titles=['FR','CETS','FORTS'],shared_yaxes=True)
df_=df[df.Market=="FR"]
fig.add_trace(go.Scatter(x=df_.date_time,y=df_.seconds
            ,name='FR'
            ,mode='markers'
            ,marker_color='rgba(152, 0, 0, .8)'
            ,hovertext='hovertext',hoverinfo=['x','y','all']
            )
        ,row=1,col=1)
df_=df[df.Market=="CETS"]
fig.add_trace(go.Scatter(x=df_.date_time,y=df_.seconds,name='CETS',mode='markers'
            #,hovertemplate=[]
    ),row=1,col=2)
df_=df[df.Market=="FORTS"]
fig.add_trace(go.Scatter(x=df_.date_time,y=df_.seconds,name='FORTS',mode='markers'),row=1,col=3)
fig.update_layout({'title':{'text':'QUIK STATISTIC','x':0.5,'y':1}})

#-----------------------------------------------------------------------------------------------
# buttons
date_buttons=[{'count':2,'step':'hour','stepmode':'todate','label':'2HRTD'}
            ,{'count':5,'step':'hour','stepmode':'backward','label':'5HBW'}
            ,{'count':10,'step':'hour','stepmode':'backward','label':'10HRTD'}
]
fig.update_layout({'xaxis':{'rangeselector':{'buttons':date_buttons}}})

#-----------------------------------------------------------------------------------------------
# drop_down buttons
# Create the buttons
dropdown_buttons = [
{'label': "ALL", 'method': "update", 'args': [{"visible": [True, True, True]}, {"title": "ALL"}]},
{'label': "FR", 'method': "update", 'args': [{"visible": [True, False, False]}, {"title": "FR"}]},
{'label': "CETS", 'method': "update", 'args': [{"visible": [False, True, False]}, {"title": "CETS"}]},
{'label': "FORTS", 'method': "update", 'args': [{"visible": [False, False, True]}, {"title": "FORTS"}]},
]
# Update the figure to add dropdown menu
fig.update_layout({
  		'updatemenus': [
        {'active': 0, 'buttons': dropdown_buttons}
        ]})
#fig.update_layout({'marker':''})
plot(fig)

'temp-plot.html'

In [151]:
fig.write_json('fig.txt')
print (os.getcwd())

c:\QUIK_for_stat\lua


#### Data Visualisation 4

In [16]:
# визуализируем данные https://plotly.com/python/reference/index/
'''
fig = make_subplots(rows=1,cols=3, subplot_titles=['FR','CETS','FORTS'],shared_yaxes=True)
col_num=1
for market in ['FR','FORTS','CETS']:
    df_=df[df.Market==market]
    fig.add_trace(go.Scatter(x=df_.date_time,y=df_.seconds,name=market,mode='markers'),row=1,col=col_num)
    col_num+=1
fig.update_layout({'title':{'text':'QUIK STATISTIC','x':0.5,'y':1}})
#fig.update_layout({'marker':''})
plot(fig)
'''
# визуализируем данные https://plotly.com/python/reference/index/
fig = make_subplots(rows=1,cols=3, subplot_titles=['FR','CETS','FORTS'],shared_yaxes=True)
# FR ________________________________________________
df_=df[(df.Market=="FR") & (df.type=='send_order')]
fig.add_trace(go.Scatter(x=df_.date_time,y=df_.seconds
            ,name='FR send_order'
            ,mode='markers'
            ,marker_color='#008000' #,marker_color='rgba(152, 0, 0, .8)'
            ,marker_symbol='square'
            ,hovertext='send_order',hoverinfo=['x','y','all']
            )
        ,row=1,col=1)
df_=df[(df.Market=="FR") & (df.type=='kill_order')]
fig.add_trace(go.Scatter(x=df_.date_time,y=df_.seconds
            ,name='FR kill_order'
            ,mode='markers'
            ,marker_color='#FF0000' # ,marker_color='rgba(100, 0, 0, .8)'
            ,marker_symbol='square-open-dot'
            ,hovertext='kill_order', hoverinfo=['x','y','all']
            )
        ,row=1,col=1)
# CETS ________________________________________________
df_=df[(df.Market=="CETS") & (df.type=='send_order')]
fig.add_trace(go.Scatter(x=df_.date_time,y=df_.seconds
            ,name='CETS send_order'
            ,mode='markers'
            ,marker_color='#800000' #,marker_color='rgba(152, 0, 0, .8)'
            ,marker_symbol='diamond'
            #,marker_size=10
            ,hovertext='send_order',hoverinfo=['x','y','all']
            )
        ,row=1,col=2)
df_=df[(df.Market=="CETS") & (df.type=='kill_order')]
fig.add_trace(go.Scatter(x=df_.date_time,y=df_.seconds
            ,name='CETS kill_order'
            ,mode='markers'
            ,marker_color='#008080' # ,marker_color='rgba(100, 0, 0, .8)'
            ,marker_symbol='diamond-open-dot'
            ,hovertext='kill_order', hoverinfo=['x','y','all']
            )
        ,row=1,col=2)
# FORTS ________________________________________________
df_=df[(df.Market=="FORTS") & (df.type=='send_order')]
fig.add_trace(go.Scatter(x=df_.date_time,y=df_.seconds
            ,name='FORTS send_order'
            ,mode='markers'
            ,marker_color='#00FFFF' #,marker_color='rgba(152, 0, 0, .8)'
            ,marker_symbol='circle'
            #,marker_size=10
            ,hovertext='send_order',hoverinfo=['x','y','all']
            )
        ,row=1,col=3)
df_=df[(df.Market=="FORTS") & (df.type=='kill_order')]
fig.add_trace(go.Scatter(x=df_.date_time,y=df_.seconds
            ,name='FORTS kill_order'
            ,mode='markers'
            ,marker_color='#FF00FF' # ,marker_color='rgba(100, 0, 0, .8)'
            ,marker_symbol='circle-open-dot'
            ,hovertext='kill_order', hoverinfo=['x','y','all']
            )
        ,row=1,col=3)

fig.update_layout({'title':{'text':'QUIK STATISTIC','x':0.5,'y':1}})
fig.update_layout({'xaxis':{'rangeslider':{'visible':True}}})
#-----------------------------------------------------------------------------------------------
# buttons
date_buttons=[{'count':2,'step':'hour','stepmode':'todate','label':'2HRTD'}
            ,{'count':5,'step':'hour','stepmode':'backward','label':'5HBW'}
            ,{'count':10,'step':'hour','stepmode':'backward','label':'10HRTD'}
]
fig.update_layout({'xaxis':{'rangeselector':{'buttons':date_buttons}}})



#fig.update_layout({'marker':''})
plot(fig)

'temp-plot.html'

#### Data Visualisation 5

In [17]:
# визуализируем данные https://plotly.com/python/reference/index/
'''
fig = make_subplots(rows=1,cols=3, subplot_titles=['FR','CETS','FORTS'],shared_yaxes=True)
col_num=1
for market in ['FR','FORTS','CETS']:
    df_=df[df.Market==market]
    fig.add_trace(go.Scatter(x=df_.date_time,y=df_.seconds,name=market,mode='markers'),row=1,col=col_num)
    col_num+=1
fig.update_layout({'title':{'text':'QUIK STATISTIC','x':0.5,'y':1}})
#fig.update_layout({'marker':''})
plot(fig)
'''
# визуализируем данные https://plotly.com/python/reference/index/
fig=go.Figure()
# FR ________________________________________________
df_=df[(df.Market=="FR") & (df.type=='send_order')]
fig.add_trace(go.Scatter(x=df_.date_time,y=df_.seconds
            ,name='FR send_order'
            ,mode='markers'
            ,marker_color='#008000' #,marker_color='rgba(152, 0, 0, .8)'
            ,marker_symbol='square'
            ,hovertext='send_order',hoverinfo=['x','y','all']
            ))
df_=df[(df.Market=="FR") & (df.type=='kill_order')]
fig.add_trace(go.Scatter(x=df_.date_time,y=df_.seconds
            ,name='FR kill_order'
            ,mode='markers'
            ,marker_color='#FF0000' # ,marker_color='rgba(100, 0, 0, .8)'
            ,marker_symbol='square-open-dot'
            ,hovertext='kill_order', hoverinfo=['x','y','all']
            ))
# CETS ________________________________________________
df_=df[(df.Market=="CETS") & (df.type=='send_order')]
fig.add_trace(go.Scatter(x=df_.date_time,y=df_.seconds
            ,name='CETS send_order'
            ,mode='markers'
            ,marker_color='#800000' #,marker_color='rgba(152, 0, 0, .8)'
            ,marker_symbol='diamond'
            #,marker_size=10
            ,hovertext='send_order',hoverinfo=['x','y','all']
            ))
df_=df[(df.Market=="CETS") & (df.type=='kill_order')]
fig.add_trace(go.Scatter(x=df_.date_time,y=df_.seconds
            ,name='CETS kill_order'
            ,mode='markers'
            ,marker_color='#008080' # ,marker_color='rgba(100, 0, 0, .8)'
            ,marker_symbol='diamond-open-dot'
            ,hovertext='kill_order', hoverinfo=['x','y','all']
            ))
# FORTS ________________________________________________
df_=df[(df.Market=="FORTS") & (df.type=='send_order')]
fig.add_trace(go.Scatter(x=df_.date_time,y=df_.seconds
            ,name='FORTS send_order'
            ,mode='markers'
            ,marker_color='#00FFFF' #,marker_color='rgba(152, 0, 0, .8)'
            ,marker_symbol='circle'
            #,marker_size=10
            ,hovertext='send_order',hoverinfo=['x','y','all']
            ))
df_=df[(df.Market=="FORTS") & (df.type=='kill_order')]
fig.add_trace(go.Scatter(x=df_.date_time,y=df_.seconds
            ,name='FORTS kill_order'
            ,mode='markers'
            ,marker_color='#FF00FF' # ,marker_color='rgba(100, 0, 0, .8)'
            ,marker_symbol='circle-open-dot'
            ,hovertext='kill_order', hoverinfo=['x','y','all']
            ))
fig.update_layout({'title':{'text':'QUIK STATISTIC','x':0.5,'y':1}})

#-----------------------------------------------------------------------------------------------
# buttons
date_buttons=[{'count':2,'step':'hour','stepmode':'todate','label':'2HRTD'}
            ,{'count':5,'step':'hour','stepmode':'backward','label':'5HBW'}
            ,{'count':10,'step':'hour','stepmode':'backward','label':'10HRTD'}
]
fig.update_layout({'xaxis':{'rangeselector':{'buttons':date_buttons}}})

#-----------------------------------------------------------------------------------------------
# drop_down buttons
# Create the buttons
dropdown_buttons = [
{'label': "ALL", 'method': "update", 'args': [{"visible": [True, True,True,True,True,True]}, {"title": "ALL"}]},

{'label': "FR send_order", 'method': "update", 'args': [{"visible": [True, False, False,False,False,False]}, {"title": "FR send_order"}]},
{'label': "FR kill_order", 'method': "update", 'args': [{"visible": [False, True, False, False, False, False]}, {"title": "FR kill_order"}]},

{'label': "CETS send_order", 'method': "update", 'args': [{"visible": [False, False, True,False,False,False]}, {"title": "CETS send_order"}]},
{'label': "CETS kill_order", 'method': "update", 'args': [{"visible": [False, False, False, True,False,False]}, {"title": "CETS kill_order"}]},

{'label': "FORTS send_order", 'method': "update", 'args': [{"visible": [False, False, False,False,True,False]}, {"title": "FORTS send_order"}]},
{'label': "FORTS kill_order", 'method': "update", 'args': [{"visible": [False, False, False, False,False,True]}, {"title": "FORTS kill_order"}]},
]
# Update the figure to add dropdown menu
fig.update_layout({
  		'updatemenus': [
        {'active': 0, 'buttons': dropdown_buttons}
        ]})


#fig.update_layout({'marker':''})
plot(fig)

'temp-plot.html'

#### Data Visualisation 6

In [18]:
# визуализируем данные https://plotly.com/python/reference/index/
fig = make_subplots(rows=1,cols=2, subplot_titles=['First','Second'],shared_yaxes=True)
fig.add_trace(go.Scatter(
                        x=df.date_time,y=df.seconds
                        ,name=''
                        ,showlegend=False
                        ,mode='markers'
                        ,marker_color=df.seconds
                        )
            ,row=1,col=1)
#fig.add_trace(go.Violin(x=df.Market,y=df.seconds,name='',orientation='v',showlegend=False),row=1,col=2)
fig.add_trace(go.Violin(
                    x=df.Market,y=df.seconds,name=''
                    ,orientation='v'
                    ,showlegend=False
                    ,box_visible=True
                    ,meanline_visible=True
                    ,line_color='black'
                    )
            ,row=1,col=2)
fig.update_layout({'title':{'text':'QUIK STATISTIC','x':0.5,'y':1}})
plot(fig)






'temp-plot.html'

In [66]:
fig=px.scatter(df,x='date_time',y='seconds',color='Market',facet_col='type')
plot(fig)

'temp-plot.html'